## Práctica SOM-MNIST

### Iván López de Munain Quintana

In [3]:
import numpy as np
import collections
import random
from sklearn import datasets
import math
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.preprocessing import minmax_scale


def euc_dist(peso, vector):
    
    delta = np.zeros(len(vector))
    
    for i in range(len(vector)):
        delta[i] = (vector[i] - peso[i])**2
        
    delta = np.sum(delta)
    delta = delta**0.5
    
    return delta

def vecindad(bmu_row, bmu_col, radio, Rows, Cols):
    
    supFila = bmu_row + radio + 1
    infFila = bmu_row - radio - 1
    supCol = bmu_col + radio + 1
    infCol = bmu_col - radio - 1
    
    mascara = np.zeros(shape=(Rows,Cols))
    
    for a in range(infFila+1, supFila):
        for b in range(infCol+1, supCol):
            mascara[a%Rows,b%Cols]=1

    return mascara


def prediction(pesos, modas, xtest, ytest):
    
    aciertos = 0
    minimo = 999
    
    for k in range(len(xtest)):
        minimo=999
        for i in range(pesos.shape[0]):
            for j in range(pesos.shape[1]):
                dist = euc_dist(pesos[i,j], xtest[k])
                #se busca la distancia minima a la neurona que tenga etiqueta (es decir, distinto de -1)
                if dist< minimo and modas[i,j]!=-1:
                    minimo = dist
                    imin=i
                    jmin=j
        
        if modas[imin,jmin]==ytest[k]:
            aciertos+=1
    
    tasa = aciertos/len(ytest)
    
    return tasa
        





In [4]:
#clase SOM

class SOM(object):
    def __init__(self,data, x, y, alpha_start=0.6, seed=42):

        np.random.seed(seed)
        self.x = x
        self.y = y
        self.shape = (x, y)
        self.radio = math.floor(x / 2)
        self.alpha_start = alpha_start
        self.epocas = 10
        self.pesos = np.random.normal(np.mean(data), np.std(data), size=(self.x, self.y, len(data[0])))
        self.permutAle = random.sample(list(np.arange(0, data.shape[0])), data.shape[0])
        

    def winner(self, vector):
        
        minimo = 9999
        for i in range(self.pesos.shape[0]):
            for j in range(self.pesos.shape[1]):
                dist = euc_dist(self.pesos[i,j], vector)
                if dist < minimo:
                    minimo = dist
                    imin = i
                    jmin = j
        
        winner = (imin,jmin)
        
        return winner


### Clasificación criterio por modas

In [6]:


# =============================== MODAS ====================================

def main():
    
    # cargar datos
    print("\nCargando datos MNIST... \n")
    
    
    
    #estos datos son una version reducida
    datos = datasets.load_digits()
    data_x = datos.data
    data_y = datos.target
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=42)
    
    x_train=minmax_scale(x_train)
    x_test=minmax_scale(x_test)
    
    '''
    #conjunto de datos completo
    dataTrain = pd.read_csv("mnist_train.csv")
    dataTest = pd.read_csv("mnist_test.csv")
    
    y_train = np.array(dataTrain.iloc[:,0])
    x_train = minmax_scale(dataTrain.iloc[:, 1:])
    
    y_test = np.array(dataTest.iloc[:,0])
    x_test = minmax_scale(dataTest.iloc[:, 1:])
    
    '''
    
    #inicializaciones
    Rows = 12
    Cols = 8
    som = SOM(x_train,Rows, Cols,alpha_start=0.6, seed=42) 
 

    

  # construccion del SOM
    print("Construccion de 12x8 SOM para MNIST")
    
    actual_radio= som.radio
    actual_alpha = som.alpha_start
    
    modas = np.full((Rows,Cols, som.epocas*len(x_train)),-1)
   
    contador = 0
    
    #epocas
    for epocas in range(som.epocas):
        
        print("\nÉpoca:",epocas)
        print("\tProcesando iteraciones...")
        
        #iteraciones
        for obs in range(len(x_train)):
            
            #if obs%10000==0:
                #print("\tIteracion:", obs)

            (bmu_row, bmu_col) = som.winner(x_train[som.permutAle[obs]])
            
            #guardado en modas que observacion (su etiqueta) esta mas cerca de que neurona
            modas[bmu_row,bmu_col,contador] = y_train[som.permutAle[obs]]
            
            
            mascara =  vecindad(bmu_row, bmu_col, actual_radio, Rows,Cols)
            
            for i in range(Rows):
                for j in range(Cols):
                    if mascara[i,j]==1:
                        som.pesos[i][j] = som.pesos[i][j] + actual_alpha * (x_train[som.permutAle[obs]] - som.pesos[i][j])
                        
            #incremento de iteracion -> actualizacion de la tasa de aprendizaje
            actual_alpha= som.alpha_start/(1+(obs/len(x_train)))
            
            contador += 1
            
        #fin de epoca -> actualizacion del radio
        actual_radio -= 1
        if actual_radio<0:
            actual_radio=0
        #som.permutAle = random.sample(list(np.arange(0, x_train.shape[0])), x_train.shape[0])
     
    
    modaNeurona = np.zeros(shape=(Rows,Cols))
    for i in range(Rows):
        for j in range(Cols):
            dic = collections.Counter(modas[i,j])
                       
            #obtenemos la moda para cada neurona
            if len(dic)==1:
                modaNeurona[i,j] = dic.most_common(1)[0][0]
            else:
                #meter aqui el comprobante de que el primero sea -1
                if dic.most_common(1)[0][0]==-1:
                    
                    modaNeurona[i,j] = dic.most_common(2)[1][0]
                
                else:
                    
                    modaNeurona[i,j] = dic.most_common(1)[0][0]
    
    
    print("Matriz de pesos construida \n")
    
    print("\nModas de las neuronas: \n")
    print(modaNeurona)
    
    tasa = prediction(som.pesos, modaNeurona, x_test,y_test)
    
    print("\n La tasa de aciertos para 12x8 SOM y el conjunto de datos MNIST es:", tasa)


    
if __name__=="__main__":
  main()



Cargando datos MNIST... 

Construccion de 12x8 SOM para MNIST

Época: 0
	Procesando iteraciones...

Época: 1
	Procesando iteraciones...

Época: 2
	Procesando iteraciones...

Época: 3
	Procesando iteraciones...

Época: 4
	Procesando iteraciones...

Época: 5
	Procesando iteraciones...

Época: 6
	Procesando iteraciones...

Época: 7
	Procesando iteraciones...

Época: 8
	Procesando iteraciones...

Época: 9
	Procesando iteraciones...
Matriz de pesos construida 


Modas de las neuronas: 

[[8. 1. 1. 1. 3. 9. 9. 8.]
 [6. 6. 6. 9. 3. 3. 3. 8.]
 [8. 6. 6. 5. 3. 3. 3. 8.]
 [6. 5. 5. 5. 3. 3. 3. 1.]
 [4. 5. 5. 5. 9. 3. 5. 1.]
 [4. 9. 9. 8. 7. 2. 1. 1.]
 [4. 4. 7. 7. 2. 1. 1. 4.]
 [4. 4. 7. 7. 7. 8. 1. 4.]
 [0. 8. 2. 2. 2. 7. 8. 0.]
 [0. 2. 2. 2. 8. 1. 0. 0.]
 [3. 2. 2. 2. 7. 8. 0. 0.]
 [5. 2. 2. 7. 9. 9. 8. 9.]]

 La tasa de aciertos para 12x8 SOM y el conjunto de datos MNIST es: 0.9494949494949495


### Clasificación por etiquetado de neuronas

In [7]:

# ============================= ETIQUETADO POR NEURONAS =============================================

def main():
    
    # carga de datos
    print("\nCargando datos MNIST... \n")
    
    
    #estos datos son una version reducida
    datos = datasets.load_digits()
    data_x = datos.data
    data_y = datos.target
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=42)
    
    x_train=minmax_scale(x_train)
    x_test=minmax_scale(x_test)
    
    '''
    #conjunto de datos completo
    dataTrain = pd.read_csv("mnist_train.csv")
    dataTest = pd.read_csv("mnist_test.csv")
    
    y_train = np.array(dataTrain.iloc[:,0])
    x_train = minmax_scale(dataTrain.iloc[:, 1:])
    
    y_test = np.array(dataTest.iloc[:,0])
    x_test = minmax_scale(dataTest.iloc[:, 1:])
    '''
    
    #inicializaciones
    Rows = 12
    Cols = 8
    som = SOM(x_train,Rows, Cols,alpha_start=0.6, seed=42) 
 

    

  # construccion SOM
    print("Construccion de 12x8 SOM para MNIST")
    
    actual_radio= som.radio
    actual_alpha = som.alpha_start

    etiqNeurona = np.zeros(shape=(Rows,Cols))
   
    contador = 0
    
    #epocas
    for epocas in range(som.epocas):
        print("\nÉpoca:",epocas)
        print("\tProcesando iteraciones...")
        #iteraciones
        for obs in range(len(x_train)):
            
            #if obs%1000==0:
                #print("\tIteracion:", obs)

            (bmu_row, bmu_col) = som.winner(x_train[som.permutAle[obs]])
                       
            mascara =  vecindad(bmu_row, bmu_col, actual_radio, Rows,Cols)
            
            for i in range(Rows):
                for j in range(Cols):
                    if mascara[i,j]==1:
                        som.pesos[i][j] = som.pesos[i][j] + actual_alpha * (x_train[som.permutAle[obs]] - som.pesos[i][j])
                        
            #incremento de iteracion -> actualizacion de la tasa de aprendizaje
            actual_alpha= som.alpha_start/(1+(obs/len(x_train)))
            
        #fin de epoca -> actualizacion del radio
        actual_radio -= 1
        if actual_radio<0:
            actual_radio=0
        #som.permutAle = random.sample(list(np.arange(0, x_train.shape[0])), x_train.shape[0])
     
    #etiquetado de neuronas
    minimo = 999
    for i in range(Rows):
        for j in range(Cols):
            minimo=999
            for t in range(len(x_train)):
                dist = euc_dist(som.pesos[i,j],x_train[t])
                if dist<minimo:
                    minimo=dist
                    tmin=t
                    
            etiqNeurona[i,j]=y_train[tmin]

    
    
    print("Matriz de pesos construida \n")
    
    print("\nEtiquetas de las neuronas: \n")
    print(etiqNeurona)
    
    tasa = prediction(som.pesos, etiqNeurona, x_test,y_test)
    
    print("\n La tasa de aciertos para 12x8 SOM y el conjunto de datos MNIST es:", tasa)


    
if __name__=="__main__":
  main()






Cargando datos MNIST... 

Construccion de 12x8 SOM para MNIST

Época: 0
	Procesando iteraciones...

Época: 1
	Procesando iteraciones...

Época: 2
	Procesando iteraciones...

Época: 3
	Procesando iteraciones...

Época: 4
	Procesando iteraciones...

Época: 5
	Procesando iteraciones...

Época: 6
	Procesando iteraciones...

Época: 7
	Procesando iteraciones...

Época: 8
	Procesando iteraciones...

Época: 9
	Procesando iteraciones...
Matriz de pesos construida 


Etiquetas de las neuronas: 

[[6. 0. 4. 7. 7. 7. 7. 6.]
 [0. 4. 4. 7. 7. 7. 7. 0.]
 [0. 0. 1. 8. 9. 9. 7. 9.]
 [3. 9. 7. 9. 9. 5. 3. 9.]
 [9. 2. 8. 5. 3. 9. 3. 3.]
 [1. 8. 2. 8. 2. 2. 2. 3.]
 [8. 4. 8. 8. 2. 8. 2. 2.]
 [4. 1. 1. 8. 4. 2. 1. 1.]
 [4. 1. 1. 5. 8. 5. 5. 4.]
 [4. 9. 2. 5. 5. 5. 4. 6.]
 [6. 9. 4. 8. 5. 3. 6. 6.]
 [6. 4. 4. 7. 1. 5. 6. 6.]]

 La tasa de aciertos para 12x8 SOM y el conjunto de datos MNIST es: 0.9511784511784511


Podemos ver cómo se obtienen muy buenos resultados con ambos criterios de clasificación (mayor del 90% de acierto) aunque se consiguen unos resultados mejores con el etiquetado de la neurona (95% de acierto). 